In [1]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import librosa
import librosa.display
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import os

In [2]:
def plot_spectrogram(sig, Y, sr, hop_length, y_axis="linear"):
    plt.figure(figsize=(10, 5))
    librosa.display.specshow(Y, sr=sr, hop_length=hop_length, x_axis="time", y_axis=y_axis)
    plt.colorbar(format="%+2.f")
    
    print(sig.shape, sr)
    
    plt.figure(figsize=(10, 5))
    plt.plot(np.linspace(0, sig.shape[0]/sr, sig.shape[0]), sig)
    plt.show()

In [3]:
class AudioClassifyDataset(Dataset):
    def __init__(self, root="./UrbanSound8K/"):
        
        self.data = []
        self.label = []
        
        FRAME_SIZE = 512
        HOP_SIZE = 512
        target_length = 88200
        
        meta = pd.read_csv(root+"/metadata/UrbanSound8K.csv")
        
        for fold in os.listdir(root+"audio/"):
            if fold[0] == '.':
                continue
            print(fold)

            for file in tqdm(os.listdir(root+"audio/"+fold+"/")):
                if file[0] == '.':
                    continue

                PATH = root+"audio/"+fold+"/"+file
                sig, sr = librosa.load(PATH)
                sig = librosa.util.fix_length(sig, target_length)

                S_scale = librosa.stft(sig, win_length=HOP_SIZE, n_fft=FRAME_SIZE, hop_length=HOP_SIZE)
                Y_scale = abs(S_scale)**2
                Y_db = librosa.power_to_db(Y_scale)
                
                
                Y_db = np.expand_dims(Y_db, axis=0)
                
                #plot_spectrogram(sig, Y_db, sr, HOP_SIZE)
                
                idx = np.where(meta['slice_file_name'] == file)[0][0]
                label = meta.iloc[[idx]]['classID'][idx]
                
                self.data.append(Y_db)
                self.label.append(label)

    def __len__(self):
        assert(len(self.data) == len(self.label))
        return len(self.label)

    def __getitem__(self, idx):
        return self.data[idx], self.label[idx]


In [4]:
data_set = AudioClassifyDataset()
torch.save(data_set, 'audio_dataset')

fold1


100%|████████████████████████████████████████████████████████████████████████████████| 874/874 [02:20<00:00,  6.24it/s]


fold10


100%|████████████████████████████████████████████████████████████████████████████████| 838/838 [02:07<00:00,  6.56it/s]


fold2


...

100%|████████████████████████████████████████████████████████████████████████████████| 839/839 [01:58<00:00,  7.10it/s]


fold8


100%|████████████████████████████████████████████████████████████████████████████████| 807/807 [02:03<00:00,  6.53it/s]


fold9


100%|████████████████████████████████████████████████████████████████████████████████| 817/817 [01:59<00:00,  6.82it/s]
